In [8]:
import numpy as np
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
import geopy.geocoders

import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

# Import Data with Full Geo Location

In [9]:
df_toronto = pd.read_csv('Toronto Geo')
df_toronto.drop('Unnamed: 0', axis = 1, inplace = True)
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Map Toronto

In [11]:
#Acquire Toronto Location
address = 'Toronto, ON'

geolocator = Nominatim(user_agent='TOexploer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Focusing on North York

In [17]:
north_york_data = df_toronto[df_toronto['Borough'] == 'North York']
north_york_data = north_york_data.reset_index(drop= True)
north_york_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493


# Map North York

In [45]:
#Acquire Toronto Location
address = 'North York, Toronto'

geolocator = Nominatim(user_agent='TOexploer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York is {}, {}.'.format(latitude, longitude))



The geograpical coordinate of North York is 43.7708175, -79.4132998.


In [25]:
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(north_york_data['Latitude'], north_york_data['Longitude'],  north_york_data['Neighbourhood']):
    
    neighborhood = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=neighborhood,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

# Get Nearby Venues 

In [51]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 2000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [52]:
CLIENT_ID = 'QTTNKNJEZE4EFKMIHQHUDEFVCYGWECPUWTQPWILPHUTEP3HP' # your Foursquare ID
CLIENT_SECRET = '02R4BIBI1X4HWRMIIJWJDOTMX1DBKTM5OHRD3LWMYMP3NSB2'# your Foursquare Secret

VERSION = '20180605' # Foursquare API version



In [53]:
print('Crawling different neighborhoods inside "North York"')
north_york_foursquare_dataset = foursquare_crawler(list(north_york_data['Postcode']),
                                                   list(north_york_data['Neighbourhood']),
                                                   list(north_york_data['Latitude']),
                                                   list(north_york_data['Longitude']),)

Crawling different neighborhoods inside "North York"
1.
Data is Obtained, for the Postal Code M2H (and Neighborhoods Hillcrest Village) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M2J (and Neighborhoods Fairview,Henry Farm,Oriole) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M2K (and Neighborhoods Bayview Village) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M2L (and Neighborhoods Silver Hills,York Mills) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M2M (and Neighborhoods Newtonbrook,Willowdale) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M2N (and Neighborhoods Willowdale South) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M2P (and Neighborhoods York Mills West) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M2R (and Neighborhoods Willowdale West) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M3A (and Neighborhoods Parkwoods) SUCCESSFULLY.
10.
Data is Obtained, for the Postal Code M3B (and Neighborhoods Don 

#  Data Wrangling

In [54]:
def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [55]:
north_york_venues = get_venue_dataset(north_york_foursquare_dataset)

Number of Venuse in Coordination "M2H" Posal Code and "Hillcrest Village" Negihborhood(s) is:
75
Number of Venuse in Coordination "M2J" Posal Code and "Fairview,Henry Farm,Oriole" Negihborhood(s) is:
100
Number of Venuse in Coordination "M2K" Posal Code and "Bayview Village" Negihborhood(s) is:
45
Number of Venuse in Coordination "M2L" Posal Code and "Silver Hills,York Mills" Negihborhood(s) is:
100
Number of Venuse in Coordination "M2M" Posal Code and "Newtonbrook,Willowdale" Negihborhood(s) is:
100
Number of Venuse in Coordination "M2N" Posal Code and "Willowdale South" Negihborhood(s) is:
40
Number of Venuse in Coordination "M2P" Posal Code and "York Mills West" Negihborhood(s) is:
100
Number of Venuse in Coordination "M2R" Posal Code and "Willowdale West" Negihborhood(s) is:
51
Number of Venuse in Coordination "M3A" Posal Code and "Parkwoods" Negihborhood(s) is:
93
Number of Venuse in Coordination "M3B" Posal Code and "Don Mills North" Negihborhood(s) is:
98
Number of Venuse in Coo

# Show Some Sample

In [57]:
north_york_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M2H,Hillcrest Village,43.803762,-79.363452,Tastee,This spot is popular,Bakery,692
1,M2H,Hillcrest Village,43.803762,-79.363452,Chatime Willowdale,This spot is popular,Bubble Tea Shop,1422
2,M2H,Hillcrest Village,43.803762,-79.363452,Bayview Golf & Country Club,This spot is popular,Golf Course,1138
3,M2H,Hillcrest Village,43.803762,-79.363452,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,This spot is popular,Korean Restaurant,754
4,M2H,Hillcrest Village,43.803762,-79.363452,Congee Queen 皇后名粥,This spot is popular,Cantonese Restaurant,1050


In [58]:
north_york_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
1810,M9M,"Emery,Humberlea",43.724766,-79.532242,Dollarama,This spot is popular,Discount Store,1693
1811,M9M,"Emery,Humberlea",43.724766,-79.532242,Dollar Tree,This spot is popular,Discount Store,1694
1812,M9M,"Emery,Humberlea",43.724766,-79.532242,Mi Pueblo,This spot is popular,Latin American Restaurant,1721
1813,M9M,"Emery,Humberlea",43.724766,-79.532242,Blue Sky,This spot is popular,Grocery Store,1774
1814,M9M,"Emery,Humberlea",43.724766,-79.532242,Shell's Food Store,This spot is popular,Grocery Store,1825


# One Hot Encoding

In [74]:

# one hot encoding
north_york_venues_onehot = pd.get_dummies(data = north_york_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
north_york_venues_onehot.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beer Store,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant,Electronics Store,Event Service,Event Space,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Music Store,Music Venue,New American Restaurant,Nightclub,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stationery Store,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M2H,Hillcrest Village,43.803762,-79.363452,Tastee,This spot is popular,692,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M2H,Hillcrest Village,43.803762,-79.363452,Chatime Willowdale,This spot is popular,1422,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

# Group by Neighbourhood

In [75]:
feature_select =  [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'American Restaurant',
 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Breakfast Spot',
 'Burger Joint',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Diner',
 'Fast Food Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 'Greek Restaurant',
 'Grocery Store',
 'Hakka Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Pizza Place',
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant',
 'Wings Joint']

In [76]:
north_york_venues_onehot = north_york_venues_onehot[feature_select].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


north_york_venues_onehot.head()

,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Greek Restaurant,Grocery Store,Hakka Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",1,2,0,0,1,0,0,0,0,1,2,0,0,1,0,1,0,0,1,0,0,0,1,0,1,4,2,3,0,1,0,0,0,0,0
Bayview Village,0,0,0,1,0,0,0,0,6,0,1,0,0,0,0,2,0,0,0,1,1,0,0,0,1,1,0,1,0,1,1,1,0,0,0
"Bedford Park,Lawrence Manor East",0,2,1,5,1,2,0,0,1,1,1,0,0,0,0,2,0,1,5,1,0,0,0,1,1,3,4,4,0,5,0,1,0,0,1
"CFB Toronto,Downsview East",3,0,0,0,0,1,0,0,0,0,2,0,0,1,1,2,0,0,0,0,0,1,1,0,1,3,2,1,0,1,0,0,0,3,0
Don Mills North,2,2,0,1,1,3,1,1,1,1,0,0,1,2,1,1,0,0,3,5,0,0,1,1,2,3,3,2,1,1,0,1,0,0,1


# Get Total Restaurant and Total Food Supply

In [77]:
feat_name_list = list(north_york_venues_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
north_york_venues_onehot['Total Restaurants'] = north_york_venues_onehot[restaurant_list].sum(axis = 1)
north_york_venues_onehot = north_york_venues_onehot.drop(columns = restaurant_list)


feat_name_list = list(north_york_venues_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
north_york_venues_onehot['Food Supply'] = north_york_venues_onehot[joint_list].sum(axis = 1)
north_york_venues_onehot = north_york_venues_onehot.drop(columns = joint_list)

# Final Dataset for K-MEANS

In [78]:
north_york_venues_onehot

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Grocery Store,Pizza Place,Sandwich Place,Total Restaurants,Food Supply
Neighborhood,,,,,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",0,1,1,0,0,1,4,3,11,1
Bayview Village,1,0,0,0,0,2,1,1,13,0
"Bedford Park,Lawrence Manor East",5,1,1,0,0,2,3,4,23,4
"CFB Toronto,Downsview East",0,0,0,0,0,2,3,1,15,2
Don Mills North,1,1,1,0,1,1,3,2,26,6
Downsview Central,1,0,0,0,0,3,4,3,12,1
Downsview Northwest,0,0,0,0,1,2,3,2,9,1
Downsview West,0,0,0,0,1,1,3,1,2,1
"Downsview,North Park,Upwood Park",2,1,0,0,0,5,4,3,21,1


In [79]:
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(north_york_venues_onehot)

In [80]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = north_york_venues_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Grocery Store,Pizza Place,Sandwich Place,Total Restaurants,Food Supply,Total Sum
G3,0.000000,0.000000,0.000000,0.000,0.000000,2.000000,2.000000,1.000000,38.000000,2.000000,45.000000
G2,1.285714,0.571429,0.571429,0.000,0.142857,2.000000,1.857143,2.714286,23.142857,3.857143,36.142857
G5,1.600000,0.600000,0.400000,0.200,0.000000,3.400000,2.800000,4.200000,20.000000,2.400000,35.600000
G1,0.375000,0.375000,0.250000,0.125,0.250000,1.500000,3.000000,2.000000,12.625000,1.375000,21.875000
G4,0.333333,0.333333,0.000000,0.000,0.333333,3.333333,1.666667,1.000000,5.000000,1.666667,13.666667


# Result

* Best Cluster is Cluster 3

* Second is Cluster 2

* Third is Cluster 5

In [83]:
north_york_summary = pd.DataFrame([north_york_venues_onehot.index, 1 + kmeans.labels_]).T
north_york_summary.columns = ['Neighborhood', 'Group']
north_york_summary

,Neighborhood,Group
0,"Bathurst Manor,Downsview North,Wilson Heights",1
1,Bayview Village,1
2,"Bedford Park,Lawrence Manor East",2
3,"CFB Toronto,Downsview East",1
4,Don Mills North,2
5,Downsview Central,1
6,Downsview Northwest,1
7,Downsview West,4
8,"Downsview,North Park,Upwood Park",5
9,"Emery,Humberlea",4


# Recommend for the Best Neighbourhood

In [84]:
north_york_summary[north_york_summary['Group'] == 3]

,Neighborhood,Group
16,"Newtonbrook,Willowdale",3
